#### 드롭아웃을 이용한 과잉적합 해소

In [ ]:
import numpy as numpy
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
# 데이터 다운로드
# 가장 많이 등장하는 상위 1000개의 단어만 선택해서 훈련데이터와 테스트 데이터를 다운로드
# 제한된 데이터 사용으로 과잉적합 원인이 된다.
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=1000)

In [ ]:
# 원-핫 인코딩으로 변환하는 함수
def one_hot_sequences(sequences, dimension=1000):
    results = numpy.zeros((len(sequences), dimension))
    for i, word_index in enumerate(sequences):
        results[i, word_index] = 1.
    return results

train_data = one_hot_sequences(train_data)  # 과잉적합을 발생시키는 원핫 인코딩
test_data = one_hot_sequences(test_data)    # 과잉적합을 발생시키는 원핫 인코딩

In [ ]:
# 신경망 모델 구축
# 드롭아웃으로 과잉적합 해소
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(1000,)))
model.add(tf.keras.layers.Dropout(0.5))   #바로 앞의 출력 16개에서 반을 Dropout
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))   #바로 앞의 출력 16개에서 반을 Dropout
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam',
                  metrics=['accuracy'])

In [ ]:
# 신경망 훈련, 검증 데이터 전달
history = model.fit(train_data,
                    train_labels,
                    epochs=20,
                    batch_size=512,
                    validation_data=(test_data, test_labels),
                    verbose=2)

In [ ]:
# 훈련 데이터의 손실값과 검증 데이터의 손실값을 그래프에 출력
history_dict = history.history
loss_values = history_dict['loss']		# 훈련 데이터 손실값
val_loss_values = history_dict['val_loss']	# 검증 데이터 손실값
acc = history_dict['accuracy']			# 정확도
epochs = range(1, len(acc) + 1)		# 에포크 수

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss Plot')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train error', 'val error'], loc='upper left')
plt.show()